# The European Football Database



## 1. Introduction

The European Football Database records data of several football championship seasons. It comes bundled as a sqlite database, which contains several tables that can easily be converted to CSV files and usable with pandas.

One has to add the target column for the challenge.

In [3]:
# !unzip data/matches.zip # has to be done once only, to comment after use

matches.zip


In [1]:
import pandas as pd 

path = 'matches.csv'
df_match = pd.read_csv(path)
df_match

,Unnamed: 0,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25977,25977,25978,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992094,7896,10243,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
import problem
X, y = problem.get_train_data()

FileNotFoundError: [Errno 2] No such file or directory: './data/matches.csv'

### Code à caser

In [ ]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.impute import SimpleImputer

import lightgbm as lgb


# Importing data

conn = sqlite3.connect('database.sqlite')
query = "SELECT * FROM Match;"

df = pd.read_sql_query(query,conn)

df = df.drop(df.columns[range(77,85)],axis='columns')

df = df.set_index('id')


# Replacing Null values

df.iloc[:,10:76] = SimpleImputer(strategy='most_frequent').fit_transform(df.iloc[:,10:76])

df = df.fillna(1)

features = list(df.columns)

for feature in features :
    print(feature, " : ", df[feature].isnull().sum())
    
# Keeping usefull features
    
X = df.drop(['home_team_goal','away_team_goal','season','date','country_id','league_id','match_api_id'],axis='columns')

# Target : 
    # 0 -> home_win, 
    # 1 -> draw, 
    # 2 -> away_win
    
Y = pd.Series(0,index=df.index)

away = df[df['home_team_goal']<df['away_team_goal']].index
draw = df[df['home_team_goal']==df['away_team_goal']].index

Y[away] = 1
Y[draw] = 2

_ = Y.value_counts().plot(kind="bar")

# Train/Test split 

train_index = df[df['season'].isin(['2008/2009','2009/2010',
                                  '2010/2011','2011/2012',
                                  '2012/2013','2013/2014'])].index

test_index = df[df['season'].isin(['2014/2015','2015/2016'])].index

X_train = X.loc[train_index.values].set_index(pd.Index(range(1,19329)))
X_test = X.loc[test_index.values].set_index(pd.Index(range(1,6652)))
y_train = Y[train_index].set_axis(pd.Index(range(1,19329)))
y_test = Y[test_index].set_axis(pd.Index(range(1,6652)))

# First model

model = lgb.LGBMClassifier()
model = model.fit(X_train,y_train)
preds = model.predict(X_test)
accuracy = sklearn.metrics.accuracy_score(y_test, preds)
print('accuracy : ', accuracy)

# Visualisation selon les variables :
    
colors = ['r', 'b', 'g']
labels = [0,1,2]

def plot_classwise_normalized(feature, bins=None):
    if bins is None:
        bins = np.linspace(X_train[feature].min(), X_train[feature].max(), 10)
    for label, color in zip(labels, colors):
        plt.hist(X_train[y_train == label][feature].values, density=True, bins=bins, 
                 alpha=0.8, color=color)
        plt.xlabel(feature)
        plt.ylabel("Density")

        
#for feature in features :
#    plt.figure()
#    plot_classwise_normalized(feature)
    
    
def plot_classwise_scatter(feature1, feature2, range1=None, range2=None):
    if range1 is None:
        range1 = [X_train[feature1].min(), X_train[feature1].max()]
    if range2 is None:
        range2 = [X_train[feature2].min(), X_train[feature2].max()]
    for label, color in zip(labels, colors):
        plt.xlim(range1[0], range1[1])
        plt.ylim(range2[0], range2[1])
        plt.scatter(X_train[y_train == label][feature1], 
                    X_train[y_train == label][feature2],
            alpha=0.3, s=80, c=color, marker='.');

## 2. Description of the columns

There are several interesting columns in the matches database.

## 3. Exploratory Data Analysis

We'll do here some plots, as seen in the Stars challenge

## 4. Explanation of the challenge

The goal is to predict the outcome of a match.

### Example of pipeline with a Random Forests classifier

In [ ]:
# code pipeline Random Forest